In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

# Preparation

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Hyperparameters
in_channel = 3
num_classes = 30
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5
loss_fn = nn.CrossEntropyLoss()

In [15]:
train_dataset = datasets.CIFAR10(root = "dataset/", train = True, transform=transforms.ToTensor(), download= True)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

Files already downloaded and verified


In [56]:
def train(dataloader, model, num_epochs, loss_fn, learning_rate):
    
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)

    losses = []

    # save model
    for epoch in range(num_epochs):
        checkpoint = {"model": model.state_dict(), "optimizer": optimizer.state_dict()}

        if epoch == 2:
            torch.save(checkpoint, "checkpoint.pth")
            print(f"saved model at epoch {epoch}")
            
        for batch, (X, y )in enumerate(dataloader):
            # to cuda
            X = X.to(device = device)
            y = y.to(device = device)

            # forward
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            # set all gradients to zero on each batch
            optimizer.zero_grad()
            loss.backward()
            # gradient descent or adam step
            optimizer.step()

            losses.append(loss.item())

            if batch % 50 == 0:
                print(f"Epoch {epoch} batch {batch} loss {losses[-1]:.4f}")
            
    return losses
    
def test(dataloader, model, loss_fn):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:

            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Load pretrained model & Modify it

In [58]:
VGG16 = torchvision.models.vgg16(pretrained = True)

/home/ossamaak0/miniconda3/envs/pytor/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ossamaak0/miniconda3/envs/pytor/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
print(VGG16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [51]:
# does nothing
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    

    def forward(self, x):
        return x

In [52]:
VGG16.avgpool = Identity()
VGG16.classifier = nn.Sequential(nn.Linear(512, 100), nn.Dropout(0.2), nn.Linear(100,10))
print(VGG16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [54]:
history = train(train_loader, VGG16.to(device), num_epochs, loss_fn, learning_rate)

Epoch 0 batch 0 loss 2.3237
Epoch 1 batch 0 loss 1.7281
saved model at epoch 2
Epoch 2 batch 0 loss 1.2167
Epoch 3 batch 0 loss 0.7336
Epoch 4 batch 0 loss 0.4305


In [57]:
test(train_loader, VGG16, loss_fn)

Test Error: 
 Accuracy: 90.9%, Avg loss: 0.279085 



# Freeze some layers (faster but maybe worse)

In [27]:
VGG16 = torchvision.models.vgg16(pretrained = True)
for param in VGG16.features.parameters():
    param.requires_grad = False

VGG16.avgpool = Identity()
VGG16.classifier = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Linear(100,10))

# history = train(train_loader, VGG16.to(device), num_epochs, loss_fn, learning_rate)
# test(train_loader, VGG16, loss_fn)

/home/ossamaak0/miniconda3/envs/pytor/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ossamaak0/miniconda3/envs/pytor/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


NameError: name 'Identity' is not defined

In [86]:
VGG16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [120]:
list(VGG16.named_parameters())[6][1].size()

torch.Size([128, 128, 3, 3])

In [121]:
list(VGG16.parameters())[6].size()

torch.Size([128, 128, 3, 3])